우리가 하려는 적대적 공격이 fgsm이라 불리는 이유. Fast Gradient Sign Method
빠른 기울기 부호 방법?
1. 빠르다.
2. 적대적 이미지를 만들기 위해 기울기의 손실을 계산하고, 기울기의 부호를 계산한다. 그리고 부호를 사용해 적대적 이미지를 생성한다.

In [2]:
# import the necessary packages
from tensorflow.keras.losses import MSE # 평균 제곱 오차. 오차를 제곱해줌!
import tensorflow as tf

def generate_image_adversary(model, image, label, eps=2/255.0):
  # 이미지 캐스트
  image = tf.cast(image,tf.float32)
  #우리의 기울기를 기록하자
  with tf.GradientTape() as tape: # 자동 차별화를 위한 작업을 기록하는 그래디언트 테이프를 생성
    #이미지에 대해 고정해야함을 명시적으로 나타낸다.
    #기울기 업데이트
    tape.watch(image)

    #우리의 모델을 입력 이미지를 예측하는데 사용하고
    #손실을 계산한다
    pred = model(image)
    loss=MSE(label,pred)

    # 이미지에 대한 손실 기울기를 계산하고
    # 기울기의 부호를 계산한다.
    gradient=tape.gradient(loss,image)
    signedGrad = tf.sign(gradient)
    #그래디언트의 부호를 취합니다. 이 코드 줄의 출력은 1,0,-1 중 하나의 값으로 채워진 벡터임.

    #적대적 이미지를 구성한다
    adversary=(image + (signedGrad * eps)).numpy() # 넘파이 배열로 변환
    #부호가 있는 그래디언트를 가져와서 작은 eps 계수를 곱함.
    #목표는 입력 이미지를 잘못 분류할 수 있을 정도의 크기지만, 인간의 눈으로는 변조를 알아차리면 안됨


    # 호출함수를 사용해 적대적 이미지를 반환
    return adversary






model - 우리가 속이려는 것

image - 넣는 이미지, 우리가 오분류하게 만드려는 것

label - 입력 이미지의 ground-truth 클래스 라벨

eps : 적대적 이미지 생성 중 입력 이미지에 적용되는 섭동의 크기를 제어함. 일반적으로 '2/255.0'과 같은 작은 값

In [3]:

# import the necessary packages
from tensorflow.keras.losses import MSE,categorical_crossentropy
import tensorflow as tf
def generate_image_adversary_target(model, image, target, eps=2 / 255.0):
	# cast the image
	image = tf.cast(image, tf.float32)
	# record our gradients
	with tf.GradientTape() as tape:
		# explicitly indicate that our image should be tacked for
		# gradient updates
		tape.watch(image)
		# use our model to make predictions on the input image and
		# then compute the loss
		pred = model(image)
		loss = categorical_crossentropy(target, pred)
	# calculate the gradients of loss with respect to the image, then
	# compute the sign of the gradient
	gradient = tape.gradient(loss, image)
	signedGrad = tf.sign(gradient)
	# construct the image adversary
	adversary = (image - (signedGrad * eps)).numpy()
	# return the image adversary to the calling function
	return adversary
